## Text Extraction

In [185]:
import openpyxl
import os
import requests

# Specify the input file path
input_file = 'Input.xlsx'

# Load the workbook and select the active worksheet
workbook = openpyxl.load_workbook(input_file)
worksheet = workbook.active

# Get the index of the columns containing the link and URL ID
link_column = None
url_id_column = None
for col_idx, cell in enumerate(worksheet[1]):
    if cell.value == 'URL':
        link_column = col_idx
    elif cell.value == 'URL_ID':
        url_id_column = col_idx
if link_column is None or url_id_column is None:
    raise ValueError('Could not find columns for link and URL ID')

if not os.path.exists('output'):
    os.makedirs('output')

# Loop over each row in the worksheet and extract the link and URL ID
for row in worksheet.iter_rows(min_row=2, values_only=True):
    try:
        link = row[link_column]
        url_id = row[url_id_column]

        # Send a GET request to the URL and save the content to a file
        response = requests.get(link)
        soup = BeautifulSoup(response.content, 'html.parser')

        # Extract the title
        title_element = soup.find('h1', {'class': 'entry-title'})
        title = title_element.text.strip()

        # Extract the text content
        article_body = soup.find('div', {'class': 'td-post-content tagdiv-type'})
        paragraphs = article_body.find_all('p')
        text = '\n'.join([p.text.strip() for p in paragraphs])

        # Save the title and text content to a file
        with open(os.path.join('output', str(url_id) + '.txt'), 'w', encoding='utf-8') as f:
            f.write(title + '\n\n' + text)
    except Exception as e:
        print(f"Error processing URL: {url}. {str(e)}")

Error processing URL: https://insights.blackcoffer.com/challenges-and-opportunities-of-big-data-in-healthcare/. 'NoneType' object has no attribute 'text'
Error processing URL: https://insights.blackcoffer.com/challenges-and-opportunities-of-big-data-in-healthcare/. 'NoneType' object has no attribute 'text'
Error processing URL: https://insights.blackcoffer.com/challenges-and-opportunities-of-big-data-in-healthcare/. 'NoneType' object has no attribute 'text'
Error processing URL: https://insights.blackcoffer.com/challenges-and-opportunities-of-big-data-in-healthcare/. 'NoneType' object has no attribute 'text'
Error processing URL: https://insights.blackcoffer.com/challenges-and-opportunities-of-big-data-in-healthcare/. 'NoneType' object has no attribute 'text'
Error processing URL: https://insights.blackcoffer.com/challenges-and-opportunities-of-big-data-in-healthcare/. 'NoneType' object has no attribute 'text'
Error processing URL: https://insights.blackcoffer.com/challenges-and-opport

Some of the links were not working(Web page not found) So I used try and except to tackle that problem

## Stopwords

In [186]:
import os

directory = r"C:\Users\User\Downloads\StopWords"
words_list = []

# Loop through all files in the directory
for filename in os.listdir(directory):
    filepath = os.path.join(directory, filename)
    if os.path.isfile(filepath):
        # Open the file and read the lines
        with open(filepath, 'r') as file:
            lines = file.readlines()
            for line in lines:
                # Split the line by whitespace and take the first word
                first_word = line.split()[0]
                # Append the first word to the list
                words_list.append(first_word)

print(words_list[:10])

['ERNST', 'YOUNG', 'DELOITTE', 'TOUCHE', 'KPMG', 'PRICEWATERHOUSECOOPERS', 'PRICEWATERHOUSE', 'COOPERS', 'AFGHANI', 'ARIARY']


Now that we have all the stop words in one list we can just use that and remove the stopwords in the text files.

In [187]:
import os
import chardet

directory = r"C:\Users\User\Downloads\output"

# Loop through all files in the directory
for filename in os.listdir(directory):
    filepath = os.path.join(directory, filename)
    if os.path.isfile(filepath):
        with open(filepath, 'rb') as f:
            encoding = chardet.detect(f.read())['encoding']
        # Open the file and read the lines
        with open(filepath, 'r',encoding=encoding) as file:
            lines = file.readlines()
        # Open the file again in write mode
        with open(filepath, 'w', encoding=encoding) as file:
            # Loop through the lines and remove words from the list
            for line in lines:
                words = line.strip().split()
                # Remove words from the list
                new_words = [word for word in words if word not in words_list]
                # Write the new line to the file
                file.write(' '.join(new_words) + '\n')

## Creating a Positive and Negative Dictionary of words

In [188]:
p_file = r"C:\Users\User\Downloads\positive-words.txt"
with open(p_file, 'r') as f:
    p = f.readlines()
positive = [item.replace('\n', '') for item in p]
positive[:10]

['a+',
 'abound',
 'abounds',
 'abundance',
 'abundant',
 'accessable',
 'accessible',
 'acclaim',
 'acclaimed',
 'acclamation']

In [189]:
n_file = r"C:\Users\User\Downloads\negative-words.txt"
with open(n_file, 'r') as f:
    n = f.readlines()
negative = [item.replace('\n', '') for item in n]
negative[:10]

['2-faced',
 '2-faces',
 'abnormal',
 'abolish',
 'abominable',
 'abominably',
 'abominate',
 'abomination',
 'abort',
 'aborted']

In [190]:
#calculating the positive score
def calculate_positive_score(text):
    words = text.split()
    return sum([1 for word in words if word in positive])

def calculate_negative_score(text):
    words = text.split()
    return sum([-1*-1 for word in words if word in negative]) 

In [191]:
directory = r"C:\Users\User\Downloads\output"
# Specify the input file path
input_file = 'Input.xlsx'
link = []
url_id = []
# Load the workbook and select the active worksheet
workbook = openpyxl.load_workbook(input_file)
worksheet = workbook.active
# results = pd.DataFrame(columns=['URL_ID', 'URL', 'POSITIVE SCORE'])
for row in worksheet.iter_rows(min_row=2, values_only=True):
    link.append(row[link_column])
    url_id.append(row[url_id_column])
#     results = results.append({'URL_ID': url_id, 'URL':link},ignore_index=True)
link = pd.DataFrame(link, columns=['URL'])
url_id = pd.DataFrame(url_id, columns=['URL_ID'])

In [192]:
positive_score = []
negative_score = []
polarity_score = []
for filename in os.listdir(directory):
    if filename.endswith('.txt'):
        filepath = os.path.join(directory, filename)
        with open(filepath, 'rb') as f:
            encoding = chardet.detect(f.read())['encoding']
        with open(filepath, 'r', encoding=encoding) as f:
            text = f.read()
            positive_score.append(calculate_positive_score(text))
            negative_score.append(calculate_negative_score(text))
#             polarity_score.append(calculate_polarity(text))
positive_score = pd.DataFrame(positive_score)
negative_score = pd.DataFrame(negative_score)
results = pd.concat([url_id, link, positive_score, negative_score], axis=1, ignore_index=True)

In [193]:
results.columns = ['URL_ID', 'URL', 'POSITIVE SCORE', 'NEGATIVE SCORE']
results.head()

,URL_ID,URL,POSITIVE SCORE,NEGATIVE SCORE
0,37.0,https://insights.blackcoffer.com/ai-in-healthc...,3.0,2.0
1,38.0,https://insights.blackcoffer.com/what-if-the-c...,23.0,37.0
2,39.0,https://insights.blackcoffer.com/what-jobs-wil...,6.0,22.0
3,40.0,https://insights.blackcoffer.com/will-machine-...,26.0,48.0
4,41.0,https://insights.blackcoffer.com/will-ai-repla...,21.0,31.0


In [194]:
results['POLARITY SCORE'] = (results['POSITIVE SCORE'] - results['NEGATIVE SCORE']) / (results['POSITIVE SCORE'] + results['NEGATIVE SCORE']) * 0.000001

In [195]:
results.head()

,URL_ID,URL,POSITIVE SCORE,NEGATIVE SCORE,POLARITY SCORE
0,37.0,https://insights.blackcoffer.com/ai-in-healthc...,3.0,2.0,2.000000e-07
1,38.0,https://insights.blackcoffer.com/what-if-the-c...,23.0,37.0,-2.333333e-07
2,39.0,https://insights.blackcoffer.com/what-jobs-wil...,6.0,22.0,-5.714286e-07
3,40.0,https://insights.blackcoffer.com/will-machine-...,26.0,48.0,-2.972973e-07
4,41.0,https://insights.blackcoffer.com/will-ai-repla...,21.0,31.0,-1.923077e-07


In [196]:
for filename in os.listdir(directory):
    if filename.endswith('.txt'):
        filepath = os.path.join(directory, filename)
        with open(filepath, 'rb') as f:
            encoding = chardet.detect(f.read())['encoding']
        with open(filepath, 'r', encoding=encoding) as f:
            text = f.read()
            words = text.split()
        results['SUBJECTIVE SCORE'] = (results['POSITIVE SCORE'] + results['NEGATIVE SCORE']) / (len(words) + 0.000001) 
results.head()

,URL_ID,URL,POSITIVE SCORE,NEGATIVE SCORE,POLARITY SCORE,SUBJECTIVE SCORE
0,37.0,https://insights.blackcoffer.com/ai-in-healthc...,3.0,2.0,2.000000e-07,0.009434
1,38.0,https://insights.blackcoffer.com/what-if-the-c...,23.0,37.0,-2.333333e-07,0.113208
2,39.0,https://insights.blackcoffer.com/what-jobs-wil...,6.0,22.0,-5.714286e-07,0.052830
3,40.0,https://insights.blackcoffer.com/will-machine-...,26.0,48.0,-2.972973e-07,0.139623
4,41.0,https://insights.blackcoffer.com/will-ai-repla...,21.0,31.0,-1.923077e-07,0.098113


In [197]:
import re
directory = r"C:\Users\User\Downloads\output"
for filename in os.listdir(directory):
    if filename.endswith('.txt'):
        filepath = os.path.join(directory, filename)
        with open(filepath, 'rb') as f:
            encoding = chardet.detect(f.read())['encoding']
        with open(filepath, 'r', encoding=encoding) as f:
            text = f.read()
            words = text.split()
            sentences = re.split(r'[.!?]+', text)
        results['AVG SENTENCE LENGTH'] = len(words)//len(sentences)
results.head()

,URL_ID,URL,POSITIVE SCORE,NEGATIVE SCORE,POLARITY SCORE,SUBJECTIVE SCORE,AVG SENTENCE LENGTH
0,37.0,https://insights.blackcoffer.com/ai-in-healthc...,3.0,2.0,2.000000e-07,0.009434,12
1,38.0,https://insights.blackcoffer.com/what-if-the-c...,23.0,37.0,-2.333333e-07,0.113208,12
2,39.0,https://insights.blackcoffer.com/what-jobs-wil...,6.0,22.0,-5.714286e-07,0.052830,12
3,40.0,https://insights.blackcoffer.com/will-machine-...,26.0,48.0,-2.972973e-07,0.139623,12
4,41.0,https://insights.blackcoffer.com/will-ai-repla...,21.0,31.0,-1.923077e-07,0.098113,12


In [198]:
def count_syllables(word):
    vowels = "aeiouy"
    count = 0
    for index in range(len(word)):
        if word[index] in vowels:
            if index == 0:
                count += 1
            elif word[index - 1] not in vowels:
                count += 1
    if word.endswith("e"):
        count -= 1
    if count == 0:
        count += 1
    return count

def find_complex_words(file_path):
    with open(file_path, 'rb') as f:
        encoding = chardet.detect(f.read())['encoding']
    with open(file_path, "r", encoding=encoding) as file:
        text = file.read()
    words = re.findall(r"\b\w+\b", text)
    complex_words = []
    for word in words:
        if count_syllables(word) > 2:
            complex_words.append(word)
    return complex_words

In [199]:
directory = r"C:\Users\User\Downloads\output"
for filename in os.listdir(directory):
    if filename.endswith('.txt'):
        filepath = os.path.join(directory, filename)
    with open(filepath, 'rb') as f:
            encoding = chardet.detect(f.read())['encoding']
    with open(filepath, 'r', encoding=encoding) as f:
            text = f.read()
            words = text.split()
    complex_words = find_complex_words(filepath)
    results['PERCENTAGE OF COMPLEX WORDS'] = len(complex_words)/len(words)

In [200]:
results['FOG INDEX'] = 0.4 * (results['AVG SENTENCE LENGTH'] + results['PERCENTAGE OF COMPLEX WORDS'])

In [201]:
directory = r"C:\Users\User\Downloads\output"
for filename in os.listdir(directory):
    if filename.endswith('.txt'):
        filepath = os.path.join(directory, filename)
        with open(filepath, 'rb') as f:
            encoding = chardet.detect(f.read())['encoding']
        with open(filepath, 'r', encoding=encoding) as f:
            text = f.read()
            words = text.split()
            sentences = re.split(r'[.!?]+', text)
        results['AVG NUMBER OF WORDS PER SENTENCE'] = len(words)//len(sentences)

In [202]:
results.head()

,URL_ID,URL,POSITIVE SCORE,NEGATIVE SCORE,POLARITY SCORE,SUBJECTIVE SCORE,AVG SENTENCE LENGTH,PERCENTAGE OF COMPLEX WORDS,FOG INDEX,AVG NUMBER OF WORDS PER SENTENCE
0,37.0,https://insights.blackcoffer.com/ai-in-healthc...,3.0,2.0,2.000000e-07,0.009434,12,0.360377,4.944151,12
1,38.0,https://insights.blackcoffer.com/what-if-the-c...,23.0,37.0,-2.333333e-07,0.113208,12,0.360377,4.944151,12
2,39.0,https://insights.blackcoffer.com/what-jobs-wil...,6.0,22.0,-5.714286e-07,0.052830,12,0.360377,4.944151,12
3,40.0,https://insights.blackcoffer.com/will-machine-...,26.0,48.0,-2.972973e-07,0.139623,12,0.360377,4.944151,12
4,41.0,https://insights.blackcoffer.com/will-ai-repla...,21.0,31.0,-1.923077e-07,0.098113,12,0.360377,4.944151,12


In [203]:
directory = r"C:\Users\User\Downloads\output"
for filename in os.listdir(directory):
    if filename.endswith('.txt'):
        filepath = os.path.join(directory, filename)
    with open(filepath, 'rb') as f:
            encoding = chardet.detect(f.read())['encoding']
    with open(filepath, 'r', encoding=encoding) as f:
            text = f.read()
            words = text.split()
    complex_words = find_complex_words(filepath)
    results['COMPLEX WORD COUNT'] = len(complex_words)

In [204]:
#removing punctuation to calculate word count
import string
directory = r"C:\Users\User\Downloads\output"
for filename in os.listdir(directory):
    if filename.endswith('.txt'):
        filepath = os.path.join(directory, filename)
    with open(filepath, 'rb') as f:
            encoding = chardet.detect(f.read())['encoding']
    with open(filepath, 'r', encoding=encoding) as f:
            text = f.read()
    text = text.translate(str.maketrans('', '', string.punctuation))
    words = text.split()
    # write the modified contents back to the file
    with open(os.path.join(directory, filename), 'w', encoding=encoding) as file:
        file.write(text)
    results['WORD COUNT'] = len(words)

In [205]:
results.head()

,URL_ID,URL,POSITIVE SCORE,NEGATIVE SCORE,POLARITY SCORE,SUBJECTIVE SCORE,AVG SENTENCE LENGTH,PERCENTAGE OF COMPLEX WORDS,FOG INDEX,AVG NUMBER OF WORDS PER SENTENCE,COMPLEX WORD COUNT,WORD COUNT
0,37.0,https://insights.blackcoffer.com/ai-in-healthc...,3.0,2.0,2.000000e-07,0.009434,12,0.360377,4.944151,12,191,530
1,38.0,https://insights.blackcoffer.com/what-if-the-c...,23.0,37.0,-2.333333e-07,0.113208,12,0.360377,4.944151,12,191,530
2,39.0,https://insights.blackcoffer.com/what-jobs-wil...,6.0,22.0,-5.714286e-07,0.052830,12,0.360377,4.944151,12,191,530
3,40.0,https://insights.blackcoffer.com/will-machine-...,26.0,48.0,-2.972973e-07,0.139623,12,0.360377,4.944151,12,191,530
4,41.0,https://insights.blackcoffer.com/will-ai-repla...,21.0,31.0,-1.923077e-07,0.098113,12,0.360377,4.944151,12,191,530


In [219]:
# directory = r"C:\Users\User\Downloads\output"
# result = []
# i = 0
# for filename in os.listdir(directory):
#     if filename.endswith('.txt'):
#         filepath = os.path.join(directory, filename)
#     with open(filepath, 'rb') as f:
#             encoding = chardet.detect(f.read())['encoding']
#     with open(filepath, 'r', encoding=encoding) as f:
#             text = f.read()
#             words = text.split()
#             for word in words:
#                 result.append(count_syllables(word))
#     results.at[i,'SYLLABLE PER WORD'] = result
#     i += 1
import nltk
nltk.download('punkt')
results['SYLLABLE PER WORD'] = ''
for filename in os.listdir(directory):
    path = os.path.join(directory, filename)
    with open(path, 'r', encoding='utf-8') as f:
        text = f.read()
        sentences = nltk.sent_tokenize(text)
        words = [nltk.word_tokenize(sentence) for sentence in sentences]
        words = [word for sublist in words for word in sublist]
        no_punct_words = [word for word in words if word.isalnum()]
        syllables_per_word = [count_syllables(word) for word in no_punct_words]
        syllables_per_word = [syl for syl in syllables_per_word if syl is not None]
        if len(syllables_per_word) > 0:
            avg_syllables = sum(syllables_per_word) / len(syllables_per_word)
        else:
            avg_syllables = 0
        url_id = file.split('_')[0]
        results.loc[results['URL_ID'] == url_id, 'avg_syllables'] = avg_syllables

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\User\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


AttributeError: '_io.TextIOWrapper' object has no attribute 'split'